# Part 5A:  Functions and Movies

In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask
import cellpose.models as models
import sutils
import plotly.graph_objs as go
import plotly.express as px

import glob

In [2]:
viewer = napari.Viewer()

# Defining Functions

We have used a lot of functions, but never written our own.  Sometimes it is handy if we are running the same thing multiple times in multiple places to have the code written only once, that way any changes/improvements made to it only need to happen once and not copy pasted at multiple locations.

First let's get an image from the worm datasets to work on.

In [28]:
file_names = glob.glob('files/*/*projection.tif')
img = ski.io.imread(file_names[0])
DAPI = img[:,:,2]
h3p = img[:,:,0]


### Thresholding function

If we find ourselves doing the same gaussian blur/threshold/object size filtering step over and over again, we might want a function we can use to do that.

In [4]:
def find_objects(input_img, blur_radius, threshold, min_size):
    # blur the image
    img = ndi.gaussian_filter(input_img, blur_radius)
    # find the threshold
    mask = img > threshold
    # label the thresholded image
    label_img, num_features = ndi.label(mask)
    # remove small objects
    label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
    return label_img

Making a function requires the 'def' keyword to show that you are defining a new function, the name of the function, parentheses, an argument list, a colon, and indentation.

Frequently you will want to return something, for instance in this case a label_img with our filtered objects.  

Now if we want to use this function on a loaded image...

In [ ]:
label_img = find_objects(DAPI, 30, 150, 1000)

viewer.layers.clear()
viewer.add_image(DAPI, name='DAPI', colormap='blue', blending='additive')
viewer.add_labels(label_img, name='segmentation', opacity=0.3, blending='additive')

<Labels layer 'segmentation' at 0x1faf4fce920>

Sometimes it is annoying trying to remember what order our input arguments are in, so we can use the argument names in our function call.  If we use the argument names, they can then be in any order.

In [10]:
label_img = find_objects(DAPI, blur_radius=30, threshold=150, min_size=1000)

viewer.layers.clear()
viewer.add_image(DAPI, name='DAPI', colormap='blue', blending='additive')
viewer.add_labels(label_img, name='segmentation', opacity=0.3, blending='additive')

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\3237256436.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  lab = ski.morphology.remove_small_objects(lab, min_size=min_size)


Note that not all arguments have to be named, if we are reasonably certain the first argument is always our image we can just leave it at as the first and not bother.

## Optional arguments

What if 9 times out of 10 you are using the same blur_radius and are tired of having to specify it?  Functions can have arguments be set to certain values by default, but once you define ONE default argument, all subsequent arguments must also have defaults so we will move it to the end.

In [12]:
def find_objects(input_img, threshold, min_size, blur_radius=30):
    # blur the image
    img = ndi.gaussian_filter(input_img, blur_radius)
    # find the threshold
    mask = img > threshold
    # label the thresholded image
    label_img, num_features = ndi.label(mask)
    # remove small objects
    label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
    return label_img

Now we can simplify our function call

In [14]:
label_img = find_objects(DAPI, threshold=150, min_size=1000)

viewer.layers.clear()
viewer.add_image(DAPI, name='DAPI', colormap='blue', blending='additive')
viewer.add_labels(label_img, name='segmentation', opacity=0.3, blending='additive')

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\1734813326.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)


<Labels layer 'segmentation' at 0x1fa8971ed40>

### Displaying as option

Now notice how it is getting annoying having to run viewer.layers.clear/add_image/add_labels every time we want to check the result?  Let's put that into the function as an option instead and make our lives easier.

We should send the napari viewer object to the function, functions should not mess with any variables that are not sent to it by arguments.

In [16]:
def find_objects(input_img, threshold, min_size, blur_radius=30, display=False, viewer=None):
    # blur the image
    img = ndi.gaussian_filter(input_img, blur_radius)
    # find the threshold
    mask = img > threshold
    # label the thresholded image
    label_img, num_features = ndi.label(mask)
    # remove small objects
    label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)

    # Optional: display the results
    if display:
        viewer.layers.clear()
        viewer.add_image(input_img, name='DAPI', colormap='blue', blending='additive')
        viewer.add_labels(label_img, name='segmentation', opacity=0.3, blending='additive')

    return label_img

In [47]:
label_img = find_objects(DAPI, threshold=150, min_size=1000, display=True, viewer=viewer)

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)


## Multiple return values

Let's do the same with a peak finding function, this time we will have it return two arguments though:  the list of peaks from ski.feature.peak_local_max and the peak_img which has 0 everywhere except at pixels that are centered on a found peak.

In [48]:
def find_peaks(input_img, sigma=8, min_distance=3, threshold_abs=0.5, display=False, viewer=None, clear=False):
    # compute the Laplacian of Gaussian
    LoG = -1000.0 * ndi.gaussian_laplace(input_img/1000.0, sigma=sigma)
    # find the peaks
    peaks = ski.feature.peak_local_max(LoG, min_distance=min_distance, threshold_abs=threshold_abs)
    # create an image with the peaks
    peak_img = np.zeros_like(LoG)
    peak_img[peaks[:,0], peaks[:,1]] = 1

    # Optional: display the results
    if display:
        if clear: 
            viewer.layers.clear()
        viewer.add_image(input_img, name='h3p', colormap='blue', blending='additive')
        viewer.add_image(LoG, name='LoG', colormap='blue', blending='additive', visible=False)
        viewer.add_points(peaks, name='peaks', size=5, opacity=0.3)
        viewer.add_image(peak_img, name='peaks_img', opacity=0.3, blending='additive')

    return peaks, peak_img



Putting both our functions together

In [50]:
label_img = find_objects(DAPI, threshold=150, min_size=1000, display=True, viewer=viewer)
pks, pk_img = find_peaks(h3p, display=True, viewer=viewer)

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)


## Multiple layers of functions

In our last class, the pipeline was to use regionprops on the results of label_img and pk_img, let's write a function to do that and return a table.

In [51]:
def get_table(DAPI, h3p, display=False, viewer=None):
    # Call our first function for segmenting worm
    label_img = find_objects(DAPI, threshold=150, min_size=1000, display=display, viewer=viewer)
    # Then our second function for finding h3p spots
    pks, pk_img = find_peaks(h3p, display=display, viewer=viewer, clear=True)

    # Combine and calculate the counts
    results = pd.DataFrame(ski.measure.regionprops_table(label_img, intensity_image=pk_img, properties=['label', 'area', 'mean_intensity']))
    results['counts'] = results['mean_intensity'] * results['area']

    return results
    

In [52]:
get_table(DAPI, h3p, display=True, viewer=viewer)

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)


,label,area,mean_intensity,counts
0,1,1594637.0,0.00016,255.0


But finally, we would really like something that takes a filename and gives us back the table.

In [53]:
def process_file(fname, display=False, viewer=None):
    img = ski.io.imread(fname)
    DAPI = img[:,:,2]
    h3p = img[:,:,0]
    return get_table(DAPI, h3p, display, viewer)


In [54]:
process_file(file_names[5], display=True, viewer=viewer)

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)


,label,area,mean_intensity,counts
0,1,5191331.0,0.000017,89.0


### Looping over files

In [56]:
all_results = []
for fname in file_names:
    current_results = process_file(fname)
    current_results['file'] = fname
    all_results.append(current_results)

df = pd.concat(all_results)

C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
C:\Users\smc\AppData\Local\Temp\ipykernel_12068\384114898.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  label_img = ski.morphology.remove_small_objects(label_img, min_size=min_size)
C:\U

Compare this with the loop we had before

In [ ]:
all_results = []

for fname in file_names:
    print(fname)
    img = ski.io.imread(fname)

    DAPI = img[:, :, 2]
    smoothed = ndi.gaussian_filter(DAPI, 30)
    thresholded = smoothed > 150
    labeled_img, object_count = ndi.label(thresholded)
    labeled_img = sutils.remove_objects(labeled_img, area_min=1000, area_max=100000000)

    h3p = img[:,:,0]
    LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=8)
    peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=0.5)
    peak_img = np.zeros_like(LoG)
    peak_img[peaks[:,0], peaks[:,1]] = 1

    results = pd.DataFrame(ski.measure.regionprops_table(labeled_img, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity']))
    results['counts'] = results['mean_intensity'] * results['area']
    
    results['file'] = fname
    all_results.append(results)

    # New stuff for saving our intermediate files
    combined_img = np.array([DAPI, labeled_img, h3p, LoG, peak_img])
    combined_img = combined_img.astype(np.single)
    output_filename = fname + 'f'
    ski.io.imsave(output_filename, combined_img,  imagej=True, metadata={'axes': 'CYX'})


By breaking our workflow into individual functions, we can troubleshoot those individually on single files and then re-use the code in the file processing loop without having to copy paste.  We also make it easier to use in the future.

# Making movies

In [69]:
viewer = napari.Viewer()

In [70]:
img = ski.io.imread('files/Neuromast.tif')

viewer.layers.clear()
viewer.add_image(img, channel_axis=2, scale=[1,0.5,.16,.16], colormap=['red', 'gray'])

[<Image layer 'Image' at 0x1fa89253340>,
 <Image layer 'Image [1]' at 0x1fa89cc2140>]

In [63]:
from napari_animation import Animation

In [67]:
nuclei = ski.data.cells3d()[:,1,...]
denoised = ndi.median_filter(nuclei, size=3)
th_nuclei = denoised > ski.filters.threshold_li(denoised)
th_nuclei = ski.morphology.remove_small_holes(th_nuclei, 20**3)
labels_data = ski.measure.label(th_nuclei)

animation = Animation(viewer)

image_layer = viewer.add_image(nuclei, name="nuclei", depiction="plane",
                               blending='translucent')
labels_layer = viewer.add_labels(labels_data, name="labels", blending='translucent')

viewer.camera.angles = (-18.23797054423494, 41.97404742075617, 141.96173085742896)
#viewer.camera.zoom *= 0.5


def replace_labels_data():
    z_cutoff = int(image_layer.plane.position[0])
    new_labels_data = labels_data.copy()
    new_labels_data[z_cutoff:] = 0
    labels_layer.data = new_labels_data


labels_layer.visible = False
image_layer.plane.position = (0, 0, 0)
animation.capture_keyframe(steps=30)

image_layer.plane.position = (59, 0, 0)
animation.capture_keyframe(steps=30)

image_layer.plane.position = (0, 0, 0)

animation.capture_keyframe(steps=30)

image_layer.plane.events.position.connect(replace_labels_data)
labels_layer.visible = True
labels_layer.experimental_clipping_planes = [{
    "position": (0, 0, 0),
    "normal": (-1, 0, 0),  # point up in z (i.e: show stuff above plane)
}]

image_layer.plane.position = (59, 0, 0)
# access first plane, since it's a list
labels_layer.experimental_clipping_planes[0].position = (59, 0, 0)
animation.capture_keyframe(steps=30)

image_layer.plane.position = (0, 0, 0)
animation.capture_keyframe(steps=30)

animation.animate("layer_planes.mp4", canvas_only=True)
image_layer.plane.position = (0, 0, 0)

Rendering frames...


100%|██████████| 121/121 [00:19<00:00,  6.13it/s]


In [60]:
img.shape

(60, 2, 256, 256)

In [61]:
viewer.add_image(img, channel_axis=1, depiction='plane')

[<Image layer 'Image' at 0x1fb00a37e80>,
 <Image layer 'Image [1]' at 0x1faf84a75e0>]